In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import eurostat
import matplotlib.pyplot as plt
import os

plt.style.use("ggplot")

%load_ext jupyter_black

# Efficiency data

## Eurostat data

### Download data

In [2]:
# df = eurostat.get_toc_df() does not work anymore
# df.head()

In [3]:
dataset_code = "NRG_BAL_C"
# df[df.code == dataset_code]

In [ ]:
list_parameters = eurostat.get_pars(dataset_code)
list_parameters

In [ ]:
dict_of_dict = {}
for parameter in list_parameters[0:]:
    print(parameter)
    list_values = eurostat.get_par_values(dataset_code, parameter)

    print(list_values)

In [6]:
# New with all year
dataset_code = "NRG_BAL_C"

# Download dictionary to rename columns with explicit names
dict_siec = eurostat.get_dic(dataset_code, "siec")
dict_siec = {key: value for (key, value) in dict_siec}

dict_nrg_bal = eurostat.get_dic(dataset_code, "nrg_bal")
dict_nrg_bal = {key: value for (key, value) in dict_nrg_bal}

In [7]:
# list_nrg_values = eurostat.get_par_values(dataset_code, "nrg_bal")
list_nrg_values = [
    "TI_EHG_MAPE_E",
    "TI_EHG_APE_E",
    "TI_EHG_MAPCHP_E",
    "GHP_MAPCHP",
    "TI_EHG_APCHP_E",
    "GHP_APCHP",
    "TI_EHG_EPS",
    "TO_EHG_PH",
    "NRG_EHG_E",
    "GEP",
    "TO_EHG",
    "DL",
]


def get_eurostat_data(year_start=2020, year_end=2023, force=False):
    """Return a xarray dataset in GWh"""
    file = f"data/ds_eurostat.nc"
    if not os.path.exists(file) or force:
        print("Downloading data, it takes a while, be patient.")
        filter_pars = {
            "freq": "A",
            "unit": "GWH",
            "startPeriod": year_start,
            "endPeriod": year_end,
            "nrg_bal": list_nrg_values,
        }
        df = eurostat.get_data_df(dataset_code, filter_pars=filter_pars)
        # df = df[["nrg_bal", "siec", "geo\TIME_PERIOD", f"{year}"]]
        df = df.drop(["freq", "unit"], axis=1)
        df = df.rename(columns={"siec": "Energies", "geo\TIME_PERIOD": "Countries"})
        df = df.pivot_table(columns=["nrg_bal", "Energies", "Countries"])
        df.index = pd.to_datetime(df.index)
        df.index.name = "DateTime"
        df = df.rename(
            columns=dict_siec, level=1
        )  # .rename(columns=dict_nrg_bal, level=0)
        ds = df.unstack().to_xarray()
        ds.to_netcdf(file)
    ds = xr.load_dataarray(file)
    return ds

In [ ]:
ds = get_eurostat_data(year_start=2020, year_end=2023, force=False)
ds

### Matching ENTSO-E - EUROSTAT names

In [9]:
def energy_matching(grid="Natural gas"):
    dict_energy_entsoe_to_eurostat = {
        "Biomass": "Bioenergy",
        "Fossil Brown coal/Lignite": "Lignite",
        "Fossil Coal-derived gas": "Manufactured gases",
        "Fossil Gas": "Natural gas",
        "Fossil Hard coal": "Anthracite",
        "Fossil Oil": "Fuel oil",
        "Fossil Oil shale": "Oil shale and oil sands",
        "Fossil Peat": "Peat",
        "Geothermal": "Geothermal",
        "Grid": grid,
        "Hydro Pumped Storage": "Hydro",
        "Hydro Run-of-river and poundage": "Hydro",
        "Hydro Water Reservoir": "Hydro",
        "Marine": "Tide, wave, ocean",
        "Nuclear": "Nuclear heat",
        "Other": "Fossil energy",
        "Other renewable": "Renewable municipal waste",
        "Solar": "Solar photovoltaic",
        "Waste": "Non-renewable waste",
        "Wind Offshore": "Wind",
        "Wind Onshore": "Wind",
    }
    return dict_energy_entsoe_to_eurostat

In [10]:
dict_energy_entsoe_to_eurostat = energy_matching()

In [11]:
from fuzzywuzzy import process

for energy_enstoe, energy_eurostat in dict_energy_entsoe_to_eurostat.items():
    if energy_eurostat not in ds.Energies.values:
        print(energy_enstoe, energy_eurostat)
        print(process.extract(energy_enstoe, ds.Energies.values))

### Calculation of efficiency

In [ ]:
# Importing data
ds_eurostat = xr.open_dataarray("data/ds_eurostat.nc")
ds_eurostat = ds_eurostat.to_dataset("nrg_bal")
ds_eurostat = ds_eurostat.sel(
    Energies=list(set(dict_energy_entsoe_to_eurostat.values()))
)
ds_eurostat = ds_eurostat.fillna(0)


# Sum of main and autoproducer of electricity only
ds_eurostat_electricity_only = (
    ds_eurostat["TI_EHG_MAPE_E"] + ds_eurostat["TI_EHG_APE_E"]
)
# Difference of CHP energy use and heat generation assuming a 85% efficiency for main produceur
ds_eurostat_CHP_main = ds_eurostat["TI_EHG_MAPCHP_E"] - ds_eurostat["GHP_MAPCHP"] / 0.85

# Difference of CHP energy use and heat generation assuming a 85% efficiency for autoproduceur
ds_eurostat_CHP_auto = ds_eurostat["TI_EHG_APCHP_E"] - ds_eurostat["GHP_APCHP"] / 0.85

# Sum of main and CHP autoproduceur
ds_eurostat_CHP = ds_eurostat_CHP_main + ds_eurostat_CHP_auto
# Sum of electricity only and CHP
ds_primary_energy_input = ds_eurostat_electricity_only + ds_eurostat_CHP
ds_primary_energy_input

In [13]:
ds_eurostat["E_pump"] = ds_eurostat["TI_EHG_EPS"] - ds_eurostat["TO_EHG_PH"]
ds_eurostat["E_own"] = ds_eurostat["NRG_EHG_E"]
ds_eurostat["E_gross"] = ds_eurostat["GEP"]
ds_eurostat["E_net"] = (
    ds_eurostat["GEP"] - ds_eurostat["NRG_EHG_E"] - ds_eurostat["E_pump"]
)

In [ ]:
ds_efficiency = ds_eurostat.E_net / ds_primary_energy_input
ds_efficiency

### Visualize efficiency data

In [15]:
import seaborn as sns

In [ ]:
ds_efficiency.ffill("DateTime").isel(DateTime=-1).drop("DateTime").squeeze()

In [ ]:
df = (
    ds_efficiency.ffill("DateTime")
    .isel(DateTime=-1)
    .drop("DateTime")
    .to_dataset("Energies")
    .to_dataframe()
    * 100
)
fig, ax = plt.subplots(figsize=(12, 4))
ax = sns.boxplot(df, ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=80)
ax.set_ylim([0, 100])
ax.set_ylabel("Plant efficiency (%)")
plt.savefig("plot/distribution_efficiency.png", bbox_inches="tight")
ax.set_title("Distribution of national fleet efficiencies by energy sources")

### Exporting data

In [ ]:
ds = xr.concat(
    [
        ds_efficiency.sel(Energies=energy_eurostat)
        for energy_enstoe, energy_eurostat in dict_energy_entsoe_to_eurostat.items()
    ],
    dim="Energies_ENTSOE",
)
ds = ds.assign_coords(
    coords={
        "Energies_ENTSOE": [
            energy_enstoe
            for energy_enstoe, energy_eurostat in dict_energy_entsoe_to_eurostat.items()
        ]
    }
)
ds = ds.rename({"Energies": "Energies_EUROSTAT"})
ds.to_netcdf("data/ds_efficiency_EUROSTAT.nc")
ds

## IEA data

### Loading data

- Data downloaded from using the script on github (not sure we can share those data)

In [ ]:
# Importing data

list_dataset = os.listdir("data/IEA")
list_dataset = [ds.split(".")[0] for ds in list_dataset]
list_ds = []
for dataset in list_dataset:
    print(dataset)
    ds = xr.load_dataarray(f"data/IEA/{dataset}.nc")
    # display(ds)
    if dataset == "ELECTRICITYANDHEAT":
        ds = ds.rename({"flowLabel": "Energies", "productLabel": "nrg_bal"})
    else:
        ds = ds.rename({"productLabel": "Energies", "flowLabel": "nrg_bal"})
    if dataset == "RENEWABLES":
        df = ds.to_dataset("Energies").to_dataframe()
        df["Biofuels"] = df[
            ["Primary solid biofuels", "Liquid biofuels", "Biogases"]
        ].sum(axis=1)
        df["Waste"] = df[["Industrial waste", "Municipal waste"]].sum(axis=1)
        df.columns.name = "Energies"
        ds = df.stack().to_xarray()
    # display(ds)
    list_ds.append(ds)

# Merging data
ds = xr.concat(list_ds, dim="Energy_Group")
ds = ds.assign_coords(coords={"Energy_Group": list_dataset})

### Matching name

In [ ]:
xr.load_dataarray(f"data/IEA/ELECTRICITYANDHEAT.nc").flowLabel.values

In [21]:
list_energies = [
    "Biofuels",
    "Coal",
    "Geothermal",
    "Hydro",
    "Natural gas",
    "Nuclear",
    "Oil",
    # "Other non-specified",
    # "Other sources",
    "Solar PV",
    "Tide",
    "Waste",
    "Wind",
]

In [22]:
def energy_matching_IEA(grid="Natural gas"):
    # Choose among energies for which we have electricity
    dict_energy_entsoe_to_IEA = {
        "Biomass": "Biofuels",
        "Fossil Brown coal/Lignite": "Coal",
        "Fossil Coal-derived gas": "Coal",
        "Fossil Gas": "Natural gas",
        "Fossil Hard coal": "Coal",
        "Fossil Oil": "Oil",
        "Fossil Oil shale": "Oil",
        "Fossil Peat": "Coal",
        "Geothermal": "Geothermal",
        "Grid": grid,
        "Hydro Pumped Storage": "Hydro",
        "Hydro Run-of-river and poundage": "Hydro",
        "Hydro Water Reservoir": "Hydro",
        "Marine": "Tide",
        "Nuclear": "Nuclear",
        "Other": "Oil",
        "Other renewable": "Solar PV",
        "Solar": "Solar PV",
        "Waste": "Waste",
        "Wind Offshore": "Wind",
        "Wind Onshore": "Wind",
    }
    return dict_energy_entsoe_to_IEA


dict_energy_entsoe_to_IEA = energy_matching_IEA()

In [23]:
from fuzzywuzzy import process

for energy_enstoe, energy_eurostat in dict_energy_entsoe_to_IEA.items():
    if energy_eurostat not in ds.Energies.values:
        print(energy_enstoe, energy_eurostat)
        print(process.extract(energy_enstoe, ds.productLabel.values))

### Calculation

In [24]:
# Energy input for electricity
ds_primary_energy_input = (
    xr.apply_ufunc(
        np.abs, ds.sel(nrg_bal=["Electricity plants", "CHP plants", "Heat plants"])
    )
    .sel(Energy_Group=["BALANCES", "RENEWABLES"])
    .sum(dim=["Energy_Group", "nrg_bal"])
    - ds.sel(nrg_bal="Heat", Energy_Group="ELECTRICITYANDHEAT") / 0.85
)
# Electricity output
ds_electricity_output = (
    ds.sel(nrg_bal="Electricity", Energy_Group="ELECTRICITYANDHEAT") * 3.6
)  # to convert GWh into TJ
ds_electricity_output = ds_electricity_output.where(ds_electricity_output > 0, np.nan)

# Efficiency
ds_efficiency = ds_electricity_output / ds_primary_energy_input

In [ ]:
ds_efficiency.sel(Energies=list_energies, country="WORLD").drop(["country"]).to_dataset(
    "Energies"
).to_dataframe().tail()

In [ ]:
dfp = (
    ds_efficiency.sel(Energies=list_energies, country="WORLD")
    .drop("country")
    .drop("Energy_Group")
    .to_dataset("Energies")
    .to_dataframe()
)
dfp[dfp < 0] = 0
dfp = dfp.dropna(axis=1, how="all")
fig, ax = plt.subplots(figsize=(6, 3))
(dfp * 100).plot(ax=ax)
plt.legend(bbox_to_anchor=(1, 1))
ax.set_ylabel("Plant efficiency (%)")
ax.set_title("Evolution of power plant efficiency\n considering heat allocation*")

In [ ]:
# Check values
energy = "Natural gas"
country = "FRA"
year = "2021"
print("Electricity output in TJ")
display(
    ds_electricity_output.sel(
        country=country, Energies=energy, year=year
    ).to_dataframe()
)
print("Energy input in TJ")
display(
    ds_primary_energy_input.sel(
        country=country, Energies=energy, year=year
    ).to_dataframe()
)
print("Efficiency")
display(ds_efficiency.sel(country=country, Energies=energy, year=year).to_dataframe())

### Visualisation

In [ ]:
import seaborn as sns

df = (
    ds_efficiency.sel(Energies=list_energies)
    .ffill("year")
    .isel(year=-1)
    .drop("year")
    .drop("Energy_Group")
    .to_dataset("Energies")
    .to_dataframe()
    * 100
)
df[df < 0] = np.nan
fig, ax = plt.subplots(figsize=(12, 4))
ax = sns.boxplot(df, ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=80)
ax.set_ylim([0, 100])
ax.set_ylabel("Plant efficiency (%)")
plt.savefig("plot/distribution_efficiency_IEA_data.png", bbox_inches="tight")
ax.set_title("Distribution of national fleet efficiencies by energy sources")

In [ ]:
df.Geothermal.describe()

In [ ]:
df[df["Natural gas"] > 100]

### Exporting data

In [ ]:
ds = xr.concat(
    [
        ds_efficiency.sel(Energies=energy_IEA)
        for energy_enstoe, energy_IEA in dict_energy_entsoe_to_IEA.items()
    ],
    dim="Energies_ENTSOE",
)
ds = ds.assign_coords(
    coords={
        "Energies_ENTSOE": [
            energy_enstoe
            for energy_enstoe, energy_IEA in dict_energy_entsoe_to_IEA.items()
        ]
    }
)
ds = ds.rename({"Energies": "Energies_IEA"})
ds.to_netcdf("data/ds_efficiency_IEA.nc")
ds

# Merge efficiency data

In [ ]:
ds_efficiency = xr.load_dataarray("data/ds_efficiency_EUROSTAT.nc")
ds_efficiency = ds_efficiency.drop("Energies_EUROSTAT")
ds_efficiency = ds_efficiency.rename({"Energies_ENTSOE": "Energies"})
ds_efficiency = ds_efficiency.rename({"Countries": "Production_Countries"})
# ds_efficiency = ds_efficiency.mean(dim="DateTime")
ds_efficiency = ds_efficiency.ffill("DateTime").isel(DateTime=-1).drop("DateTime")
df_efficiency_eurostat = ds_efficiency.to_dataset("Energies").to_dataframe()
df_efficiency_eurostat.head()

In [ ]:
ds_efficiency = xr.load_dataarray("data/ds_efficiency_IEA.nc")
ds_efficiency = ds_efficiency.drop("Energy_Group").drop("Energies_IEA")
ds_efficiency = ds_efficiency.ffill("year").isel(year=-1).drop("year")
ds_efficiency = ds_efficiency.rename({"Energies_ENTSOE": "Energies"})
ds_efficiency = ds_efficiency.rename({"country": "Production_Countries"})
df_efficiency_IEA = ds_efficiency.to_dataset("Energies").to_dataframe()
df_efficiency_IEA.head()

In [34]:
# List ENTSOE countries
list_countries = [
    "AL",
    "AM",
    "AT",
    "AZ",
    "BA",
    "BE",
    "BG",
    "BY",
    "CH",
    "CZ",
    "DE",
    "DK",
    "EE",
    "ES",
    "FI",
    "FR",
    "GB",
    "GE",
    "GR",
    "HR",
    "HU",
    "IE",
    "IT",
    "LT",
    "LU",
    "LV",
    "MD",
    "ME",
    "MK",
    "MT",
    "NL",
    "NO",
    "PL",
    "PT",
    "RO",
    "RS",
    "RU",
    "SE",
    "SI",
    "SK",
    "TR",
    "UA",
    "XK",
]

In [ ]:
import pycountry


def convert_to_ISO2(country):
    if country == "UNK":
        country = "XK"
    if len(country) == 3:
        country = pycountry.countries.get(alpha_3=country).alpha_2
    return country


df_efficiency_IEA.index = [
    convert_to_ISO2(country) for country in df_efficiency_IEA.index
]
df_efficiency_IEA.head()

In [ ]:
# Completing the EUROSTAT data with IEA data when data are missing
df_efficiency = df_efficiency_eurostat.combine_first(df_efficiency_IEA)
df_efficiency[df_efficiency < 0] = np.nan
df_efficiency.columns.name = "Energies"
df_efficiency.index.name = "Production_Countries"
ds_efficiency = df_efficiency.stack().to_xarray()
ds_efficiency.to_netcdf("data/ds_efficiency_combined.nc")
ds_efficiency

In [37]:
def fill_efficiency_missing_data(self, ds, filling_quantile=0.10):
    # Missing countries
    list_missing_countries = list(
        set(self.ds_consumption_tracked.Production_Countries.values)
        - set(ds.Production_Countries.values)
    )
    print("Missing countries:")
    print(list_missing_countries)
    df = ds.to_dataset("Energies").to_dataframe()
    # Removing negative values due to missing data
    df[df < 0] = np.nan
    # Add missing countries
    df_missing = pd.DataFrame(columns=df.columns, index=list_missing_countries)
    df_missing.index.name = df.index.name
    df = pd.concat([df, df_missing])
    df.columns.name = "Energies"
    df = df.fillna(df.quantile(filling_quantile))
    ds = df.unstack().to_xarray()
    ds = ds.sel(
        Production_Countries=self.ds_consumption_tracked.Production_Countries.values
    )
    return ds


def get_efficiency_data(self, filling_quantile=0.1):
    """Return the efficiency dataset based on EUROSTAT data"""
    ds_efficiency = xr.load_dataarray("data/ds_efficiency.nc")
    ds_efficiency = ds_efficiency.drop("Energies_EUROSTAT")
    ds_efficiency = ds_efficiency.rename({"Energies_ENTSOE": "Energies"})
    ds_efficiency = ds_efficiency.rename({"Countries": "Production_Countries"})
    ds_efficiency = ds_efficiency.mean(dim="DateTime")
    ds_efficiency = self.fill_efficiency_missing_data(
        ds=ds_efficiency, filling_quantile=filling_quantile
    )
    return ds_efficiency

# Emissions factor

In [ ]:
# Importing data from emission_factors_energy_products
df_fuels = pd.read_csv(
    "data/emission_factors_energy_products.csv",
    header=[0, 1],
    index_col=1,
    delimiter=";",
)
df_fuels.head()

In [ ]:
# Extracting emmission factor
df_ef = df_fuels["kg/TJ"]
df_ef.head()
df_ef.columns = [col.replace("ef", "") for col in df_ef.columns]
df_ef.head()

In [ ]:
# Calculating the equivalent carbon footprint
GWP_factors = {"CO2": 1, "CH4": 25, "N2O": 298}
df_ef_CO2eq = df_ef.multiply(GWP_factors)
df_ef_CO2eq["Total"] = df_ef_CO2eq.sum(axis=1)

# Concatenating all data together
df_ef_CO2eq = pd.concat([df_ef_CO2eq], axis=1, keys=["kg CO2eq / TJ (combustion)"])
df_fuels = pd.concat([df_fuels, df_ef_CO2eq], axis=1)
df_fuels.head()

In [ ]:
dfp = pd.DataFrame()
dfp["Upstream"] = df_fuels["kg CO2eq / TJ"]["UpstreamE3"]
dfp["Combustion"] = df_fuels["kg CO2eq / TJ (combustion)"]["Total"]

fig, ax = plt.subplots(figsize=(8, 12))
ax2 = ax.twiny()
(dfp * 1e3 / 1e6).plot(ax=ax, kind="barh", stacked=True, alpha=0.5)
(dfp / 1e6 * 3.6 * 1e3).plot(ax=ax2, kind="barh", stacked=True, alpha=0.5)
ax.set_title("Observation of life-cycle emissions")
ax.set_xlabel("gCO2eq / MJ_primary_energy")
ax2.set_xlabel("gCO2eq / kWh_primary energy")
ax2.grid(False)

In [ ]:
df = dfp / 1e6 * 3.6 * 1e3
df.loc["Fossil energy"] = df.loc["Fuel oil"]
df.loc["Bioenergy"] = df.loc["Biogases"]

df.index.name = "Energies_EUROSTAT"
df.columns.name = "Emission_phase"
ds_EF = df.unstack().to_xarray()
ds_EF

In [44]:
ds = xr.concat(
    [
        ds_EF.sel(Energies_EUROSTAT=energy_eurostat)
        for energy_enstoe, energy_eurostat in dict_energy_entsoe_to_eurostat.items()
    ],
    dim="Energies_ENTSOE",
)
ds = ds.assign_coords(
    coords={
        "Energies_ENTSOE": [
            energy_enstoe
            for energy_enstoe, energy_eurostat in dict_energy_entsoe_to_eurostat.items()
        ]
    }
)
ds.to_netcdf("data/ds_emissions_factors.nc")

In [ ]:
ds

# Losses

In [46]:
# list_nrg_values = eurostat.get_par_values(dataset_code, "nrg_bal")
list_loss_values = ["LOSS", "TL", "TRANSL", "DL", "DL_NT", "TRANSL_DL"]

In [ ]:
for col in list_loss_values:
    print(f"{col}: {dict_nrg_bal[col]}")

In [48]:
def get_eurostat_loss_data(year_start=2020, year_end=2023, force=False):
    """Return a xarray dataset in GWh"""
    file = f"data/ds_eurostat_loss.nc"
    if not os.path.exists(file) or force:
        print("Downloading data, it takes a while, be patient.")
        filter_pars = {
            "freq": "A",
            "unit": "GWH",
            "startPeriod": year_start,
            "endPeriod": year_end,
            "nrg_bal": ["NEP", "LOSS", "DL", "DL_NT", "TRANSL"],
        }
        df = eurostat.get_data_df("nrg_cb_e", filter_pars=filter_pars)
        # df = df[["nrg_bal", "siec", "geo\TIME_PERIOD", f"{year}"]]
        df = df.drop(["freq", "unit"], axis=1)
        df = df.rename(columns={"siec": "Energies", "geo\TIME_PERIOD": "Countries"})
        df = df.pivot_table(columns=["nrg_bal", "Energies", "Countries"])
        df.index = pd.to_datetime(df.index)
        df.index.name = "DateTime"
        df = df.rename(
            columns=dict_siec, level=1
        )  # .rename(columns=dict_nrg_bal, level=0)
        ds = df.unstack().to_xarray()
        ds = ds / ds.sel(nrg_bal="NEP")
        ds = ds.sel(Energies="Electricity")
        ds.to_netcdf(file)
    ds = xr.load_dataarray(file)
    return ds

In [ ]:
ds = get_eurostat_loss_data(year_start=2020, year_end=2023, force=False)
ds

In [ ]:
ds.sel(nrg_bal="TRANSL").ffill("DateTime").isel(DateTime=-1).to_dataframe("gl")

## Plot of grid lossess

In [55]:
from ecodynelec.statistics_LF import *

df = (
    ds.sel(nrg_bal="TRANSL", DateTime="2022")
    .drop_vars(["nrg_bal", "Energies", "DateTime"])
    .to_dataframe("grid_losses")
)
df = df.droplevel(1, 0)
gdf = get_geodraframe(df * 100)

In [ ]:
gdf.head()

In [ ]:
fig, ax = plt.subplots()
gdf.plot(
    "grid_losses",
    ax=ax,
    cmap="Blues",
    edgecolor="k",
    legend=True,
    # vmin=0,
    # vmax=1,
    legend_kwds={
        "location": "bottom",
        "label": "Grid losses (%)",
    },
)
ax.set_xlim([2e6, 6e6])
ax.set_ylim([1.4e6, 5e6])

ax.grid(False)
ax.axis("off")

In [ ]:
df.describe()

In [ ]:
country = "IT"
ds.sel(Countries=country).plot(hue="nrg_bal")
ds.sel(Countries=country).to_dataset("nrg_bal").to_dataframe()

In [ ]:
df = (
    ds.sel(DateTime="2022")
    .drop_vars(["Energies", "DateTime"])
    .to_dataset("nrg_bal")
    .to_dataframe()
)
df = df.droplevel(1, 0)
gdf = get_geodraframe(df * 100)
gdf.head()

In [ ]:
# Almost no data for non technical distribution losses (i.e. thief)
gdf.DL_NT.replace(0, np.nan).dropna()

In [ ]:
fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(16, 6))
gdf.plot(
    "DL",
    ax=ax[0],
    cmap="Blues",
    edgecolor="k",
    legend=True,
    vmin=0,
    vmax=18,
    legend_kwds={
        "location": "bottom",
        "label": "Grid losses (%)",
    },
)
ax[0].grid(False)
ax[0].axis("off")
ax[0].set_title("Distribution losses")

gdf.plot(
    "TRANSL",
    ax=ax[1],
    cmap="Blues",
    edgecolor="k",
    legend=True,
    vmin=0,
    vmax=18,
    legend_kwds={
        "location": "bottom",
        "label": "Grid losses (%)",
    },
)
ax[1].grid(False)
ax[1].axis("off")
ax[1].set_title("Transport losses")

gdf.plot(
    "LOSS",
    ax=ax[2],
    cmap="Blues",
    edgecolor="k",
    legend=True,
    vmin=0,
    vmax=18,
    legend_kwds={
        "location": "bottom",
        "label": "Grid losses (%)",
    },
)
ax[2].grid(False)
ax[2].axis("off")
ax[2].set_title("T&D losses")

ax[0].set_xlim([2e6, 6e6])
ax[0].set_ylim([1.4e6, 5e6])